In [17]:
%load_ext autoreload
%autoreload 2
# %load_ext wurlitzer

import sys
sys.path.append("../")

import numpy as np
import time
from utils import model
from inference import log_marginal_likelihood
from inference import logmarglik
from inference import zstates as zs
from inference import zstates_old_method as zs_old
from utils import hyperparameters
from iotools import snp_annotator

nsample = 300
nsnps = 100
pi = 0.1
mu = 0.0
sigmabg = 0.000001
sigma = 0.1
tau = 1 / (0.005 * 0.005)

x, y, csnps, v = model.simulate(nsample, nsnps,
                                pi = pi,
                                mu = mu,
                                sigma = sigma,
                                sigmabg = sigmabg,
                                tau = tau)

nvar = x.shape[0]
nsample = x.shape[1]
distances = np.linspace(-40000, 40000,nsnps)
dist_params = [-5, 20, -20, 0.038]
# dfeat = snp_annotator.myExpcurveMix(distances/d0, *dist_params)
dfeat = snp_annotator.myTcurve(distances)
dist_feature = 1/dfeat
# dist_feature = np.ones(nsnps)

feat1 = np.ones((nvar, 1))
feat2 = np.random.rand(nvar)[np.newaxis].T
features = np.concatenate((feat1, feat2), axis=1)
# features = feat1
nfeat = features.shape[1]

# gamma0 = - np.log((1 / pi) - 1)
# gamma0 = pi
gammaN = [-2 for i in range(0,nfeat)]
gammaN[0] = - np.log((1 / pi) - 1)
params = np.array( gammaN + [mu, sigma, sigmabg, tau])
scaledparams = hyperparameters.scale(params)
cmax = 1
zstates_new = zs.create(scaledparams, x, y, features, dist_feature, cmax, nvar, 0.98)
# zstates_old = zs_old.create(scaledparams, x, y, features, cmax, nvar, 0.98)
# zstates_new = zstates_old

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.0 0.98
zstates.py: Working with 2 leading zstates.


In [18]:
%load_ext autoreload
%autoreload 2

zstates = zstates_new
# params = np.array([-2,  0, 0.5, 0.0680367, 1 / 0.001 / 0.001])
_mu = mu
_sigma = sigma + 0.01
_sigmabg = sigmabg + 0.0000001
_tau = tau + 0.01


gammaN = [-2 for i in range(0,nfeat)]
gammaN[0] = - np.log((1 / pi) - 1)
params = np.array( gammaN + [_mu, _sigma, _sigmabg, _tau])
print(params)

scaledparams = hyperparameters.scale(params)
success, m, der = logmarglik.func_grad(scaledparams, x, y, features, dist_feature, zstates)

print("logmL, der:", m, der)

# Derivatives of gamma
for i in range(0, nfeat):
    delta = 1e-8
    newparams = hyperparameters.scale(params)
    newparams[i] += delta
    newm = logmarglik.func(newparams, x, y, features, dist_feature, zstates)
    gamma_grad = (newm - m) / delta
    print ("Derivative of gamma {:d} by brute force is {:f}".format(i, gamma_grad))
    print ("Derivative of gamma {:d} from equation is {:f}".format(i, der[i]))

# Derivative of mu
delta = 1e-8
newparams = hyperparameters.scale(params)
newparams[nfeat + 0] += delta
newm = logmarglik.func(newparams, x, y, features, dist_feature, zstates)
mu_grad = (newm - m) / delta
print ("Derivative of mu by brute force is {:f}".format(mu_grad))
print ("Derivative of mu from equation is {:f}".format(der[nfeat + 0]))

# Derivative of sigma
delta = 1e-7
newparams = hyperparameters.scale(params)
newparams[nfeat + 1] += delta
newm = logmarglik.func(newparams, x, y, features, dist_feature, zstates)
sigma_grad = (newm - m) / delta
print ("Derivative of sigma by brute force is {:f}".format(sigma_grad))
print ("Derivative of sigma from equation is {:f}".format(der[nfeat + 1]))

# Derivative of sigbg
delta = 1e-7
newparams = hyperparameters.scale(params)
newparams[nfeat + 2] += delta
newm = logmarglik.func(newparams, x, y, features, dist_feature, zstates)
sigbg_grad = (newm - m) / delta
print ("Derivative of sigbg by brute force is {:f}".format(sigbg_grad))
print ("Derivative of sigbg from equation is {:f}".format(der[nfeat + 2]))

# Derivative of tau
delta = 1e-8
newparams = hyperparameters.scale(params)
newparams[nfeat + 3] += delta
newm = logmarglik.func(newparams, x, y, features, dist_feature, zstates)
tau_grad = (newm - m) / delta
print ("Derivative of tau by brute force is {:f}".format(tau_grad))
print ("Derivative of tau from equation is {:f}".format(der[nfeat + 3]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[-2.19722458e+00 -2.00000000e+00  0.00000000e+00  1.10000000e-01
  1.10000000e-06  4.00000100e+04]
logmL, der: 412963.9854821429 [ 4.07874077e+00  8.80274215e-01  1.38609166e-01 -1.32472412e+00
 -2.11305890e+01 -8.28203538e+05]
Derivative of gamma 0 by brute force is 4.074536
Derivative of gamma 0 from equation is 4.078741
Derivative of gamma 1 by brute force is 0.878936
Derivative of gamma 1 from equation is 0.880274
Derivative of mu by brute force is 0.133878
Derivative of mu from equation is 0.138609
Derivative of sigma by brute force is 1.000008
Derivative of sigma from equation is -1.324724
Derivative of sigbg by brute force is -21.063606
Derivative of sigbg from equation is -21.130589
Derivative of tau by brute force is -827586.499508
Derivative of tau from equation is -828203.537907


In [3]:
der

array([ 6.01278400e-01, -3.45141035e-01, -9.82245266e-02, -1.67235135e-01,
       -1.40751243e-01, -6.51852364e+05])

In [ ]:
params = np.array([0.00495222, 0, 0.48975, 0.0680367, 1 / 0.00001 / 0.00001])
scaledparams = hyperparameters.scale(params)

start_time = time.time()
l, m = log_marginal_likelihood.full(scaledparams, x, y, zstates_old)
print ("Log marginal likelihood from full calculation: {:f}".format(m))
print("Calculated in {:f} seconds ---\n".format(time.time() - start_time))

start_time = time.time()
m = log_marginal_likelihood.func(scaledparams, x, y, zstates_old)
print ("Log marginal likelihood from fast calculation: {:f}".format(m))
print("Calculated in {:f} seconds ---\n".format(time.time() - start_time))

start_time = time.time()
m = log_marginal_likelihood.func(scaledparams, x, y, zstates_old)
print ("Log marginal likelihood using old method of zstates: {:f}".format(m))
print("Calculated in {:f} seconds ---\n".format(time.time() - start_time))

In [ ]:
zstates = zstates_new
#params = np.array([0.01, 0.1, 0.0003, 1.0, 1 / 0.5 / 0.5])
# params = np.array([0.00495222, 0, 0.48975, 0.0680367, 1 / 0.0001 / 0.0001])
params = np.array([pi, _mu, _sigma, _sigmabg, _tau])
scaledparams = hyperparameters.scale(params)
m, der = log_marginal_likelihood.func_grad(scaledparams, x, y, zstates)

# Derivative of pi
delta = 1e-6
newparams = hyperparameters.scale(params)
newparams[0] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
pi_grad = (newm - m) / delta
print ("Derivative of pi by brute force is {:f}".format(pi_grad))
print ("Derivative of pi from equation is {:f}".format(der[0]))

# Derivative of mu
delta = 1e-8
newparams = hyperparameters.scale(params)
newparams[1] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
mu_grad = (newm - m) / delta
print ("Derivative of mu by brute force is {:f}".format(mu_grad))
print ("Derivative of mu from equation is {:f}".format(der[1]))

# Derivative of sigma
delta = 1e-8
newparams = hyperparameters.scale(params)
newparams[2] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
sigma_grad = (newm - m) / delta
print ("Derivative of sigma by brute force is {:f}".format(sigma_grad))
print ("Derivative of sigma from equation is {:f}".format(der[2]))

# Derivative of sigbg
delta = 1e-8
newparams = hyperparameters.scale(params)
newparams[3] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
sigbg_grad = (newm - m) / delta
print ("Derivative of sigbg by brute force is {:f}".format(sigbg_grad))
print ("Derivative of sigbg from equation is {:f}".format(der[3]))

# Derivative of tau
delta = 1e-8
newparams = hyperparameters.scale(params)
newparams[4] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
tau_grad = (newm - m) / delta
print ("Derivative of tau by brute force is {:f}".format(tau_grad))
print ("Derivative of tau from equation is {:f}".format(der[4]))